In [ ]:
# PREDICTIVE MAINTENANCE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_auc_score
import joblib

data = pd.read_csv('/content/predictive_maintenance.csv')
data.dropna()
data.head()

features = data[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']]
features.head()


labels = data[['Failure Type']]
labels.head()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels

joblib.dump(label_encoder, 'label_encoder.pkl')

def save_model(model, path='lstm_model.pth'):

    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')

def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(model.fc.weight.device), targets.to(model.fc.weight.device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total

    print("Confusion Matrix:")
    print(confusion_matrix(all_targets, all_predictions))

    print("\nClassification Report:")
    print(classification_report(all_targets, all_predictions))

    print(f'\nTest Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# PREVENTIVE MAINTENANCE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

data = pd.read_csv('/content/predictive_maintenance.csv')
data.dropna()
data.head()

features = data[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']]
features.head()

timestamps = pd.date_range(start='2023-01-01', periods=len(features), freq='H', name='Timestamps [hrs]')
timestamps = pd.to_datetime(timestamps, errors='coerce')
timestamps

features.loc[:, 'Timestamps [hrs]'] = timestamps
features = features.set_index('Timestamps [hrs]')
features.head(11)

def create_sequences(df, seq_length):
    X, y = [], []

    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i+seq_length].values

        X.append(seq)
        y.append(df.iloc[i+seq_length].values)

    return np.array(X), np.array(y)

seq_length = 10

X, y = create_sequences(features, seq_length)
X, y

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X = scaler_x.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
y = scaler_y.fit_transform(y.reshape(-1, y.shape[-1])).reshape(y.shape)

X.shape, y.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

batch_size = 64

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

input_size = X_train.shape[2]
hidden_size = 64
num_layers = 2
output_size = y_train.shape[1]

model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)
model.to(device)
model

def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    test_losses = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(model.fc.weight.device), targets.to(model.fc.weight.device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            test_losses.append(loss.item())

    avg_loss = test_loss / len(test_loader)

    print(f'\nTest Loss: {avg_loss:.4f}')

def save_model(model, path='lstm_model_v3.pth'):
    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')

<ipython-input-4-9b65167456de>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.loc[:, 'Timestamps [hrs]'] = timestamps


In [ ]:
model.eval()
sample_input = X_test[0].clone().detach().to(device).unsqueeze(0).float()

with torch.no_grad():
    output = model(sample_input)

output = output.cpu().numpy()
output = scaler_y.inverse_transform(output.reshape(1, -1))[0]

sample_input = sample_input.squeeze(0).cpu().numpy()
sample_input = scaler_x.inverse_transform(sample_input)

sample_input, sample_input.shape

(array([[ 302.1     ,  310.9     , 1666.      ,   30.7     ,  203.      ],
        [ 302.1     ,  311.      , 1364.      ,   50.      ,  205.      ],
        [ 302.1     ,  311.      , 1351.      ,   60.3     ,  207.      ],
        [ 302.1     ,  311.      , 1729.      ,   26.1     ,  210.      ],
        [ 302.1     ,  311.      , 1574.      ,   34.1     ,  215.      ],
        [ 302.      ,  311.      , 1566.      ,   37.8     ,  218.      ],
        [ 302.      ,  310.9     , 1455.      ,   43.2     ,  220.      ],
        [ 302.      ,  310.9     , 1679.      ,   29.      ,  222.      ],
        [ 302.      ,  310.8     , 2143.      ,   17.300001,  227.      ],
        [ 302.      ,  310.8     , 1447.      ,   42.7     ,  229.      ]],
       dtype=float32),
 (10, 5))

In [ ]:
!pip install pyngrok

In [ ]:
!pip install hugchat

In [ ]:
!pip install flask-cors

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
import torch.nn as nn
import numpy as np
import joblib
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)
port = 5000

!ngrok authtoken  2lY6u1RoJRj2asr21C0gcztCIe7_25iccpu5Gxs5TbP79wnVk
public_url = ngrok.connect(port)
print("Ngrok Tunnel URL:", public_url)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class PreventiveLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PreventiveLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class PredictiveLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(PredictiveLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        return out

preventive_model = PreventiveLSTM(input_size=5, hidden_size=64, num_layers=2, output_size=5)
predictive_model = PredictiveLSTM(input_size=5, hidden_size=64, num_layers=2, num_classes=6)

preventive_model.load_state_dict(torch.load('lstm_model_prediction.pth', map_location=device))
predictive_model.load_state_dict(torch.load('lstm_model_classification.pth', map_location=device))

preventive_model.to(device).eval()
predictive_model.to(device).eval()

scaler_x = joblib.load('scaler_x.pkl')
scaler_y = joblib.load('scaler_y.pkl')
label_encoder = joblib.load('label_encoder.pkl')

def preprocess_input(input_data):
    input_array = np.array(input_data).reshape(1, -1)
    input_scaled = scaler_x.transform(input_array)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32, device=device).unsqueeze(0)
    return input_tensor

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        features = data['features']
        input_tensor = preprocess_input(features)

        with torch.no_grad():

            preventive_output = preventive_model(input_tensor)
            predictive_input_tensor = torch.tensor(preventive_output, dtype=torch.float32, device=device).unsqueeze(0)
            predictive_output = predictive_model(predictive_input_tensor)

        _, predicted_class = torch.max(predictive_output, 1)
        output = label_encoder.inverse_transform(predicted_class.cpu().numpy()).item()
        failure_type = output

        if failure_type == 'No Failure':
            insights = "By processing the past 10 hours of data, the model predicts that the machine is not likely to fail in the 11th hour."
        else:
            command = f"Provide insights and recommendations about {failure_type} occurring in machines"
            insights = processCommand(command)

        return jsonify({
            'predicted_failure_type': output,
            'insights': insights   })

    except Exception as e:
        print(f"Error in prediction: {e}")
        return jsonify({'error': str(e)}), 500

from hugchat import hugchat

def chatBot(query):
    user_input = query.lower()
    chatbot = hugchat.ChatBot(cookie_path="huggingface.json")
    id = chatbot.new_conversation()
    chatbot.change_conversation(id)   response = chatbot.chat(user_input)
    print(f"Insights: {response}")
    return response

def processCommand(command):
    if not command:
        print("No command received.")
        return ""

    try:
        response = chatBot(command + ". give solution in a short and effectively understandable format")
        if not isinstance(response, str):
            response = str(response)
        print(f"Recommendations: {response}")
        return response
    except Exception as e:
        print(f"Error in processing the command: {e}")
        return ""

if __name__ == '__main__':
    app.run(port=port)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Ngrok Tunnel URL: NgrokTunnel: "https://a2df-35-204-134-202.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-9-88b8dcf41b86>:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  preventive_model.load_state_dict(torch.load('lstm_model_prediction.pth', map_location=device

Insights: **Random Failures in Machines: Insights and Recommendations**

**What are Random Failures?**

Random failures are unexpected and unpredictable equipment failures that occur without warning, often due to wear and tear, human error, or external factors.

**Common Causes:**

1. **Aging**: Wear and tear over time
2. **Human Error**: Incorrect maintenance, operation, or repair
3. **Environmental Factors**: Temperature, humidity, vibration, or contamination
4. **Design or Manufacturing Flaws**: Inherent weaknesses in the machine's design or manufacturing process

**Consequences:**

1. **Downtime**: Lost productivity and revenue
2. **Repair Costs**: Unexpected expenses for parts and labor
3. **Safety Risks**: Potential harm to personnel and equipment

**Recommendations:**

1. **Regular Maintenance**: Schedule routine maintenance to identify and address potential issues before they become major problems.
2. **Condition-Based Maintenance**: Use sensors and monitoring systems to detect

INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 09:57:04] "OPTIONS /predict HTTP/1.1" 200 -
<ipython-input-9-88b8dcf41b86>:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictive_input_tensor = torch.tensor(preventive_output, dtype=torch.float32, device=device).unsqueeze(0)
INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 09:57:31] "POST /predict HTTP/1.1" 200 -


Insights: Here are some insights and recommendations about random failures occurring in machines:

**Understanding Random Failures**

Random failures, also known as sporadic or intermittent failures, are unexpected and unpredictable failures that occur without a clear pattern or warning. They can be caused by various factors, including:

1. **Component degradation**: Wear and tear on machine components can lead to sudden failures.
2. **Electrical issues**: Power surges, voltage fluctuations, or electrical noise can cause machines to malfunction.
3. **Software glitches**: Bugs or errors in software code can lead to unexpected failures.
4. **Human error**: Operator mistakes or incorrect maintenance procedures can cause machines to fail.
5. **Environmental factors**: Temperature, humidity, or vibration can affect machine performance.

**Recommendations to Minimize Random Failures**

1. **Regular Maintenance**: Schedule regular maintenance to detect and replace worn-out components.
2. **Mo

...

INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 10:01:15] "POST /predict HTTP/1.1" 200 -


Insights: Here are some insights and recommendations about random failures occurring in various systems, presented in a concise and easily understandable format:

**Causes of Random Failures:**

1. **Wear and Tear**: Gradual deterioration of components due to usage and aging.
2. **Human Error**: Mistakes made by operators, maintenance personnel, or designers.
3. **Design Flaws**: Inherent weaknesses in the system design or architecture.
4. **Environmental Factors**: External conditions such as temperature, humidity, and vibrations.
5. **Manufacturing Defects**: Flaws in the manufacturing process or materials.

**Recommendations to Minimize Random Failures:**

1. **Regular Maintenance**: Schedule regular checks and maintenance to detect and replace worn-out components.
2. **Training and Education**: Provide operators and maintenance personnel with proper training and education to reduce human error.
3. **Design for Reliability**: Incorporate reliability-centered design principles to min

INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 10:01:23] "POST /predict HTTP/1.1" 200 -


Insights: Here are some insights and recommendations about random failures occurring in various systems, presented in a concise and easily understandable format:

**Insight 1: Understand the Root Cause**

* **Problem:** Random failures can be caused by various factors, including human error, equipment malfunction, or environmental factors.
* **Recommendation:** Identify the root cause of the failure by analyzing data, conducting investigations, and consulting with experts. This will help you address the underlying issue and prevent future failures.

**Insight 2: Monitor and Maintain Equipment**

* **Problem:** Poorly maintained equipment can lead to random failures.
* **Recommendation:** Regularly inspect and maintain equipment, perform routine checks, and replace worn-out parts to prevent failures.

**Insight 3: Implement Redundancy and Backup Systems**

* **Problem:** Single-point failures can cause significant disruptions.
* **Recommendation:** Implement redundant systems, backup po

INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 10:19:33] "OPTIONS /predict HTTP/1.1" 200 -
<ipython-input-9-88b8dcf41b86>:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictive_input_tensor = torch.tensor(preventive_output, dtype=torch.float32, device=device).unsqueeze(0)
INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 10:19:34] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 10:20:08] "OPTIONS /predict HTTP/1.1" 200 -
<ipython-input-9-88b8dcf41b86>:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictive_input_tensor = torch.tensor(preventive_output, dtype=torch.float32, device=device).unsqueeze(0)
INFO:werkzeug:127.0.0.1 - - [04/Sep/2024 10:20:08] "POST /predict HT